<style> h1 {text-align: center;}</style>
<h1>Email Analysis </h1>


## Email Headers

| Field                          | Details                                                                                                                                            |
|--------------------------------|----------------------------------------------------------------------------------------------------------------------------------------------------|
| From                           | The sender's address.                                                                                                                              |
| To                             | The receiver's address, including CC and BCC.                                                                                                      |
| Date                           | Timestamp, when the email was sent.                                                                                                                |
| Subject                        | The subject of the email.                                                                                                                          |
| Return Path                    | The return address of the reply, a.k.a. "Reply-To".If you reply to an email, the reply will go to the address mentioned in this field.             |
| Domain Key and DKIM Signatures | Email signatures are provided by email services to identify and authenticate emails.                                                               |
| SPF                            | Shows the server that was used to send the email.It will help to understand if the actual server is used to send the email from a specific domain. |
| Message-ID                     | Unique ID of the email.                                                                                                                            |
| MIME-Version                   | Used MIME version. It will help to understand the delivered "non-text" contents and attachments.                                                   |
| X-Headers                      | The receiver mail providers usually add these fields.Provided info is usually experimental and can be different according to the mail provider.    |
| X-Received                     | Mail servers that the email went through.                                                                                                          |
| X-Spam Status                  | Spam score of the email.                                                                                                                           |
| X-Mailer                       | Email client name.                                                                                                                                 |

---    

## Important Email Header Fields for Quick Analysis
| Questions to Ask / Required Checks&nbsp;                      | Evaluation                                                                                                    |
|---------------------------------------------------------------|---------------------------------------------------------------------------------------------------------------|
| Do the "From", "To", and "CC" fields contain valid addresses? | Having invalid addresses is a red flag.                                                                       |
| Are the "From" and "To" fields the same?                      | Having the same sender and recipient is a red flag.                                                           |
| Are the "From" and "Return-Path" fields the same?             | Having different values in these sections is a red flag.                                                      |
| Was the email sent from the correct server?                   | Email should have come from the official mail servers of the sender.                                          |
| Does the "Message-ID" field exist, and is it valid?           | Empty and malformed values are red flags.                                                                     |
| Do the hyperlinks redirect to suspicious/abnormal sites?      | Suspicious links and redirections are red flags.                                                              |
| Do the attachments consist of or contain malware?             | Suspicious attachments are a red flag.File hashes marked as suspicious/malicious by sandboxes are a red flag. |
---     

## Email Inspection Tooling

### Text Editor
You can used a text editor to view the message. However, their might be some limitations, especially when an email has attachments.

### `emlAnalyzer`
You can use this to do an in depth analysis. It will parse the email and separate it in sections.

### https://emailrep.io/ 
Use this OSINT tool to check the reputation of the addresses found in the sender and return path.

---     

## Further OSINT Analysis of Email Artifacts
| Tool             | Purpose                                                                                                                |
|------------------|------------------------------------------------------------------------------------------------------------------------|
| VirusTotal       | A service that provides a cloud-based detection toolset and sandbox environment.                                       |
| InQuest          | A service provides network and file analysis by using threat analytics.                                                |
| IPinfo.io        | A service that provides detailed information about an IP address by focusing on geolocation data and service provider. |
| Talos Reputation | An IP reputation check service is provided by Cisco Talos.                                                             |
| Urlscan.io       | A service that analyses websites by simulating regular user behaviour.                                                 |
| Browserling      | A browser sandbox is used to test suspicious/malicious links.                                                          |
| Wannabrowser     | A browser sandbox is used to test suspicious/malicious links.                                                          |

---     

# Solving the task

# Quick look using the Sublime Text Editor
I first took a quick look at `Urgent:.eml` file in Sublime

## Things that stood out + Making sense of the email
From top to bottom:
- It looks like the email was sent internal per the `X-Pnm-Origin` header...
- This was sent from **chief.elf@santaclaus.thm**
    - This could be the account that was compromised
- `Content-Type: multipart/mixed;boundary=----------03edd9c682aec8f60d54b9e4bb86659f`
    - It looks like this email is broken down in multiple parts (hence the multipart)
    - We can continue to follow the chain below
- It was sent to **elves.all@santaclaus.thm**
    - This could be an email group which includes all users who are elves
    - Thus, it was probably sent out to all the elves. Which means their is a decent chance that at least one fell for the email.
- Their is a file attached: "Division_of_labour-Load_share_plan.doc"
    - This could be a malicious file, possibly with some macro magic 🤔
    - Or instructing the victim to do something...?
- `Message-Id` is Base64 encoded...
    - We can decode this using `echo '<base64 content>' | base64 --decode
- It has a Spamscore of **3**
- In the `Received` header we can see that it was indeed sent internally from and by mail.santaclaus.thm.
- The `Return-Path` email looks a bit suspicious: **murphy.evident@bandityeti.thm**
- Following the multipart content:
    - We see the `----------03edd9c682aec8f60d54b9e4bb86659f` from the initial Content-Type header above.
        - It looks like this is the start of that content this header was referring to
    - This follows to another `Content-Type:multipart/ related; boundary=----------8f117c48beda7f1c1da0a5a894d5c4b5`, which in turn is followed by the start of this `8f117c48beda7f1c1da0a5a894d5c4b5` content
        - This one however has a content-type of text/html in utf-8, it also contained some base64 content
    - It is also followed by the next Content-Type of "application/msword", which is a base64 encoded version of the attachment. Base64 allows us to transfer binary data easily through http or just about any protocol as text based data.
    - From the `Content-Disposition` following the filename/name is the actual file attachment.
        - If we follow it all the way to the end, we see this is where the actual payload for the original Content-Type header of `03edd9c682aec8f60d54b9e4bb86659f` ends, signaled by the double dashes `--` following it: `03edd9c682aec8f60d54b9e4bb86659f--`

## Decoded the message using base64
I grabbed the base64 contenxt of the first `8f117c48beda7f1c1da0a5a894d5c4b5` and decoded it
```Shell
echo '<InsertBase64 Here>' | base64 --decode
```
This is done automatically for you with the **emlAnalyzer** tool below in the HTML section of the output.

## Extracting the attached file manually
We can snag the document by copying the base64 following the name of the file and then decoding to an output file. Something like this:
```Shell
echo '<InsertBase64 Here>' | base64 --decode > output.doc
```
This is also done automatically with the `--extract-all` flag in **emlAnalyzer**

# Using the `emlAnalyzer tool
I ran the following command:
```Shell
emlAnalyzer -i Urgent\:.emt --header --html -u --text --extract-all
```

This automatically decodes the Base64 HTML of the message and it also extracts the attached "doc" file into the eml_attachments directory.

---    

To answer some of the follow up questions:
To get the hash of the doc:
`sha256sum <name of file>`

Then simply use VirusTotal and InQuest to get the details!

## HAPPY HUNTING :)
